# Validation ROC AUC: 0.74242

In [ ]:
!pip install numpy==1.26.4
!pip install pandas==2.2.2
!pip install scikit-learn==1.3.2
!pip install catboost==1.2.7

In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm
import multiprocessing
import gc

# 병렬 처리를 위한 CPU 코어 수 설정
N_JOBS = multiprocessing.cpu_count() - 1  # 1개는 시스템용으로 남김

#메모리 관리를 위한 가비지 컬렉션 함수
def clean_memory():
    gc.collect()

# 가중치 적용 함수 수정
def apply_feature_weights(X, weight_dict):
    X_weighted = X.copy()
    for column in X.columns:
        if column in weight_dict["IVF"]:
            # 카테고리형 변수 체크 방식 수정
            if isinstance(X[column].dtype, pd.CategoricalDtype):
                continue
            X_weighted[column] *= weight_dict["IVF"][column]
    return X_weighted
# 0. 데이터 변환 함수들은 동일하게 유지
def convert_count_str(val):
    if pd.isna(val):
        return 0.0
    val = str(val).strip()
    if "회 이상" in val:
        return 6.0
    m = re.search(r'(\d+)', val)
    if m:
        return float(m.group(1))
    return 0.0

# Progress Bar를 포함한 데이터 전처리 함수
def preprocess_data(df, category_columns, is_train=True):
    print("데이터 전처리 시작...")
    
    # tqdm으로 진행률 표시
    for col in tqdm(count_columns, desc="횟수 관련 컬럼 변환"):
        df[col] = df[col].astype(str).apply(convert_count_str)
    
    # 나이 관련 변환
    print("나이 관련 데이터 변환 중...")
    df['시술 당시 나이_missing'] = df['시술 당시 나이'].apply(
        lambda x: 1.0 if str(x).strip() == '알 수 없음' else 0.0)
    df['시술 당시 나이'] = df['시술 당시 나이'].map(age_mapping).fillna(0)
    
    # 기증자 나이 변환
    df['난자 기증자 나이'] = df['난자 기증자 나이'].astype(str).apply(convert_donor_age)
    df['정자 기증자 나이'] = df['정자 기증자 나이'].astype(str).apply(convert_donor_age)
    
    # 카테고리 변환
    print("카테고리 데이터 변환 중...")
    df = convert_nan_to_string(df, category_columns)
    for col in category_columns:
        df[col] = df[col].astype("category")
    
    return df



# 메인 실행 코드
if __name__ == "__main__":
    # 메모리 초기 정리
    clean_memory()
    
    print("데이터 로딩 중...")
    train = pd.read_csv('train.csv').drop(columns=['ID'])
    test = pd.read_csv('test.csv').drop(columns=['ID'])
    weight_data = pd.read_csv('NEW_final_ivf_di_weighted_data.csv')
    weight_dict = weight_data.set_index("데이터 항목").to_dict()
    
    category_columns = [
        "시술 시기 코드", "시술 유형", "특정 시술 유형", "배란 유도 유형",
        "배아 생성 주요 이유", "IVF 임신 횟수", "DI 임신 횟수",
        "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처"
    ]
    
    count_columns = ["총 임신 횟수", "총 출산 횟수", "총 시술 횟수", 
                    "IVF 시술 횟수", "DI 시술 횟수", "클리닉 내 총 시술 횟수"]
    
    print("학습 데이터 전처리 중...")
    X = preprocess_data(train.drop('임신 성공 여부', axis=1), category_columns)
    y = train['임신 성공 여부']
    del train
    clean_memory()
    
    print("테스트 데이터 전처리 중...")
    X_test = preprocess_data(test, category_columns, is_train=False)
    del test
    clean_memory()
    
    print("가중치 적용 중...")
    X_weighted = apply_feature_weights(X, weight_dict)
    X_test_weighted = apply_feature_weights(X_test, weight_dict)
    del X, X_test
    clean_memory()
    
    print("데이터 리샘플링 중...")
    undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
    X_resampled, y_resampled = undersample.fit_resample(X_weighted, y)
    del X_weighted
    clean_memory()
    
    print("데이터 분할 중...")
    X_train, X_val, y_train, y_val = train_test_split(
        X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
    )
    del X_resampled, y_resampled
    clean_memory()
    
    print("모델 구성 중...")
    cat_est = CatBoostClassifier(
        verbose=0,
        iterations=150,
        learning_rate=0.05,
        thread_count=4
    )
    cat_est.set_params(cat_features=get_categorical_feature_indices(X_train))
    
    stack_clf = StackingClassifier(
        estimators=[
            ('xgb', XGBClassifier(
                tree_method='hist',
                enable_categorical=True,  # 카테고리형 변수 지원 활성화
                n_estimators=150,
                max_depth=6,
                learning_rate=0.05,
                n_jobs=4
            )),
            ('lgbm', LGBMClassifier(
                n_jobs=4,
                verbose=0,
                n_estimators=150,
                learning_rate=0.05,
                categorical_feature=category_columns  # LGBM 카테고리 지정
            )),
            ('cat', cat_est)
        ],
        final_estimator=Pipeline([
            ('scaler', StandardScaler()),
            ('lr', LogisticRegression(max_iter=2000, n_jobs=4))
        ]),
        cv=5,
        n_jobs=1
    )
    
    # 파라미터 그리드 유지
    param_grid = {
        'xgb__n_estimators': [150],
        'xgb__max_depth': [6],
        'xgb__learning_rate': [0.05],
        'lgbm__n_estimators': [150],
        'lgbm__learning_rate': [0.05],
        'cat__iterations': [150],
        'cat__learning_rate': [0.05],
        'final_estimator__lr__C': [1.0]
    }
    
    print("그리드 서치 시작...")
    grid_search = GridSearchCV(
        stack_clf,
        param_grid,
        scoring='roc_auc',
        cv=5,
        n_jobs=1,
        verbose=2
    )
    
    # 모델 학습
    grid_search.fit(X_train, y_train)
    
    print("\n검증 성능 평가 중...")
    y_val_pred = grid_search.best_estimator_.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_val_pred)
    print(f"Validation ROC AUC: {auc:.5f}")
    
    print("최종 예측 생성 중...")
    pred_proba = grid_search.best_estimator_.predict_proba(X_test_weighted)[:, 1]
    
    submission = pd.DataFrame({
        'ID': [f"TEST_{i:05d}" for i in range(len(X_test_weighted))],
        'probability': pred_proba
    })
    submission.to_csv('final_weighted_submission.csv', index=False)
    print("최종 제출 파일 'final_weighted_submission.csv' 생성 완료")

In [ ]:
# 검증 세트에 대한 ROC-AUC 산출
y_val_pred = grid_search.best_estimator_.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)
print(f"Validation ROC AUC: {roc_auc:.5f}")